In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
!pip install -Uq transformers datasets torch numpy psutil pynvml

In [3]:
import torch, transformers, datasets
import numpy as np
torch.__version__, transformers.__version__, datasets.__version__, np.__version__

('2.1.2+cu121', '4.36.2', '2.16.1', '1.26.3')

In [4]:
import os
import psutil
import time
import gc
from pynvml import *

from threading import Thread


def print_process_memory():
    process = psutil.Process(os.getpid())
    
    memory_info = process.memory_info()
    memory_usage = memory_info.rss / (1024 ** 2) 
    
    gc.collect()
    
    memory_info2 = process.memory_info()
    memory_usage2 = memory_info2.rss / (1024 ** 2) 
    
    print(f"Current memory usage: {memory_usage:>10.1f} MB -> {memory_usage2:>10.1f} MB")

def print_gpu_utilization():
    torch.cuda.empty_cache() 
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def schedule_gpu_memory_logging():
    def log_loop():
        while True:
            #print_process_memory()
            print_gpu_utilization()
            time.sleep(30)

    t = Thread(target=log_loop, daemon=True)
    t.start()
schedule_gpu_memory_logging()

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, set_seed
from torch import nn
from torch.nn import functional as F
import math

hf_ckp = 'distilbert-base-uncased'
hf_ckp = 'roberta-base'
set_seed(100)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {f"accuracy": (predictions == labels).mean()}

def count_parameters(m, verbose=True):
    total_count = 0
    learnable_count = 0
    if verbose:
        print("Parameters (name, tunable, count):")

    output_width = max([len(n) for n, _ in m.named_parameters()])
    for n, p in m.named_parameters():
        count = p.data.numel()
        if verbose:
            print(f" {n:{output_width}} {p.requires_grad:5b} {count:>11d}")
        total_count += count
        if p.requires_grad:
            learnable_count += count

    print(
        f"Total parameters: {total_count:,}, "
        f"thereof learnable: {learnable_count:,} "
        f"({learnable_count/total_count*100.:5.4f}%)"
    )

    return total_count, learnable_count

def adapt_model(model):
    
    # Minimalized example in place of the original LoRA-from-Scratch 
    # implementation from the article: 
    # https://towardsdatascience.com/dive-into-lora-adapters-38f4da488ede
    class MinimalLoRAAdapter(nn.Module): 
        def __init__(self, 
                     adaptee):
            super().__init__()

            self.adaptee = adaptee

            self.orig_forward = adaptee.forward
            adaptee.forward = self.forward
            
            r = 1
            adaptee.lora_A = nn.Parameter(
                torch.randn(adaptee.in_features, r) / math.sqrt(adaptee.in_features)
            )
            adaptee.lora_B = nn.Parameter(torch.zeros(r, adaptee.out_features))

        def forward(self, x, *args, **kwargs):
            return (
                self.orig_forward(x, *args, **kwargs)
                + F.dropout(x, 0.1) @ self.adaptee.lora_A @ self.adaptee.lora_B
            )

    if False:
        for p in list(model.children()):
            p.requires_grad_(False)
        for p in list(model.children())[-1].parameters():
            p.requires_grad_(True)

   
    if True:
        # freeze all layers, incl. embeddings, except for the classifier
        for m in model.roberta.modules():    
        #for m in model.distilbert.modules():    
            m.requires_grad_(False)

    # Adapt linear modules in transformer layers
    for m in model.roberta.encoder.modules():    
    #for m in model.distilbert.transformer.modules():    
        if isinstance(m, nn.Linear):
            MinimalLoRAAdapter(m)
    return model

GPU memory occupied: 516 MB.


2024-01-17 12:05:34.566909: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
%%time

tokenizer = AutoTokenizer.from_pretrained(hf_ckp)
collator = DataCollatorWithPadding(tokenizer=tokenizer)

datasets.logging.disable_progress_bar()
dataset = datasets.load_dataset("glue", "sst2")
train = dataset["train"]
valid = dataset["validation"]

def preprocess_function(examples):
        return tokenizer(examples['sentence'], padding=False, truncation=True)

tokenized_train = train.map(preprocess_function, batched=False)
tokenized_valid = valid.map(preprocess_function, batched=False)

def train(cp_enabled, model):
    
    set_seed(100)
    #torch.random.set_rng_state(torch.ByteTensor(100))
    if cp_enabled:
        model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant":False})
    
    training_args = TrainingArguments(
        gradient_checkpointing=cp_enabled,
        output_dir="out",    
        per_device_train_batch_size=224,
        learning_rate=3e-5,
        save_steps=10_000,
        eval_steps=   250,
        max_steps = 1_000,
        evaluation_strategy="steps",
        save_strategy="steps",
        save_total_limit=1,
        disable_tqdm=True,
        metric_for_best_model='eval_accuracy',
        report_to="none", # Disable wandb, tensorboard
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_valid,
        tokenizer=tokenizer,
        data_collator=collator,
        compute_metrics=compute_metrics,
    )
    print(f'{model.is_gradient_checkpointing=}')
    total, learnable = count_parameters(model, verbose=False)
    
    trainer.train()
    trainer.evaluate()


print('\n---- without gradient checkpointing ----\n')
model = AutoModelForSequenceClassification.from_pretrained(hf_ckp, num_labels=2)   
model = adapt_model(model)
train(False, model)

del(model) # essential!

print('\n---- with gradient checkpointing ----\n')
model = AutoModelForSequenceClassification.from_pretrained(hf_ckp, num_labels=2)
model = adapt_model(model)

train(True, model)


---- without gradient checkpointing ----



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.is_gradient_checkpointing=False
Total parameters: 124,813,058, thereof learnable: 758,018 (0.6073%)


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


GPU memory occupied: 1541 MB.
GPU memory occupied: 1441 MB.
GPU memory occupied: 1419 MB.
GPU memory occupied: 2283 MB.
{'eval_loss': 0.33385390043258667, 'eval_accuracy': 0.8841743119266054, 'eval_runtime': 1.6844, 'eval_samples_per_second': 517.688, 'eval_steps_per_second': 64.711, 'epoch': 0.83}
GPU memory occupied: 1447 MB.
GPU memory occupied: 6321 MB.
GPU memory occupied: 6305 MB.
{'loss': 0.4722, 'learning_rate': 1.5e-05, 'epoch': 1.66}
{'eval_loss': 0.2571539282798767, 'eval_accuracy': 0.9013761467889908, 'eval_runtime': 1.7027, 'eval_samples_per_second': 512.126, 'eval_steps_per_second': 64.016, 'epoch': 1.66}
GPU memory occupied: 1431 MB.
GPU memory occupied: 1431 MB.
GPU memory occupied: 4887 MB.
GPU memory occupied: 1421 MB.
{'eval_loss': 0.2381727248430252, 'eval_accuracy': 0.908256880733945, 'eval_runtime': 1.6827, 'eval_samples_per_second': 518.211, 'eval_steps_per_second': 64.776, 'epoch': 2.49}
GPU memory occupied: 2845 MB.
GPU memory occupied: 1399 MB.
GPU memory occu

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.is_gradient_checkpointing=True
Total parameters: 124,813,058, thereof learnable: 758,018 (0.6073%)


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


GPU memory occupied: 1381 MB.
GPU memory occupied: 1381 MB.
{'eval_loss': 0.6846632361412048, 'eval_accuracy': 0.5091743119266054, 'eval_runtime': 1.6905, 'eval_samples_per_second': 515.83, 'eval_steps_per_second': 64.479, 'epoch': 0.83}
GPU memory occupied: 1381 MB.
GPU memory occupied: 1381 MB.
{'loss': 0.6764, 'learning_rate': 1.5e-05, 'epoch': 1.66}
{'eval_loss': 0.6755141615867615, 'eval_accuracy': 0.5091743119266054, 'eval_runtime': 1.6976, 'eval_samples_per_second': 513.656, 'eval_steps_per_second': 64.207, 'epoch': 1.66}
GPU memory occupied: 1381 MB.
GPU memory occupied: 1381 MB.
{'eval_loss': 0.6665772199630737, 'eval_accuracy': 0.5103211009174312, 'eval_runtime': 1.7146, 'eval_samples_per_second': 508.585, 'eval_steps_per_second': 63.573, 'epoch': 2.49}
GPU memory occupied: 1381 MB.
GPU memory occupied: 1381 MB.
{'loss': 0.6586, 'learning_rate': 0.0, 'epoch': 3.32}
{'eval_loss': 0.6635248064994812, 'eval_accuracy': 0.5194954128440367, 'eval_runtime': 1.6695, 'eval_samples_per

In [7]:
# Doesn't matter that it is LoRA. Using a Custom Module. Freezing layers. All of that happens in the adapt_model method. 
# Missing LoRA, missing require_grad_ adjustments
# Adapted from my real example, article.

GPU memory occupied: 1381 MB.
GPU memory occupied: 1381 MB.
GPU memory occupied: 1381 MB.
GPU memory occupied: 1381 MB.
GPU memory occupied: 1381 MB.
GPU memory occupied: 1381 MB.
GPU memory occupied: 1381 MB.
GPU memory occupied: 1381 MB.
